In [1]:
from keras.preprocessing import image
from keras_preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
from keras.layers import normalization, AveragePooling2D, Dense, Input, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications import vgg16
from keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import layers
import tensorflow as tf



## Variaveis

In [10]:
# Variaveis:

# Pré-processamento
batch_size = 8
img_height = 180
img_width = 180
epochs=10
init_LR = 1e-3

#Rede

# VGG19
# Compile



## Pré-Processamento

In [2]:
Image_gen = ImageDataGenerator(rescale = 1/255, shear_range=10, zoom_range=0.3, horizontal_flip=True, vertical_flip=True,brightness_range=[0.5,2.0], width_shift_range = 0.2,rotation_range=20,fill_mode = 'nearest')
val_Datagen = ImageDataGenerator(rescale = 1/255)

In [3]:
train = Image_gen.flow_from_directory('train',batch_size=32, class_mode='binary',target_size=(224,224,3))

validation = Image_gen.flow_from_directory('val',batch_size=2,class_mode='binary',target_size=(224,224,3))

test = val_Datagen.flow_from_directory('test',batch_size=2,class_mode='binary',target_size=(224, 224, 3))
# criando as classes como variave

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Rede

In [4]:
 # colocando todas as layers da vgg19
def vgg_layers():
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False

    x = vgg.output          #outputs = [vgg.get_layer(name).output for name in layer_names]
    x = GlobalAveragePooling2D()(x)
    x = Dense(128,activation='relu')(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=vgg.input, outputs=predictions)        #    model = tf.keras.Model([vgg.input],outputs)
    return model




In [5]:
model = vgg_layers()

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

## Callbacks

In [7]:
#early stopping
stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20) #verbose=2, patience=10

# Learing rete
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=8)

## Compile

In [16]:
# Compilando
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy']) #loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

## Treinamento

In [13]:

#history = model.fit(train, validation_data=test, epochs=epochs, verbose=2)
history = model.fit(train,epochs=30, validation_data=validation, steps_per_epoch=100, callbacks=[stopper,lr], batch_size=32)

Epoch 1/10
131/131 - 33s - loss: 0.2545 - accuracy: 0.8961 - val_loss: 0.9855 - val_accuracy: 0.7356 - 33s/epoch - 249ms/step
Epoch 2/10
131/131 - 34s - loss: 0.1080 - accuracy: 0.9584 - val_loss: 0.8511 - val_accuracy: 0.7708 - 34s/epoch - 257ms/step
Epoch 3/10
131/131 - 31s - loss: 0.0927 - accuracy: 0.9632 - val_loss: 1.0821 - val_accuracy: 0.7468 - 31s/epoch - 233ms/step
Epoch 4/10
131/131 - 31s - loss: 0.0611 - accuracy: 0.9785 - val_loss: 1.3615 - val_accuracy: 0.7548 - 31s/epoch - 233ms/step
Epoch 5/10
131/131 - 32s - loss: 0.0557 - accuracy: 0.9792 - val_loss: 1.6091 - val_accuracy: 0.7356 - 32s/epoch - 247ms/step
Epoch 6/10
131/131 - 32s - loss: 0.0340 - accuracy: 0.9881 - val_loss: 1.7704 - val_accuracy: 0.7484 - 32s/epoch - 241ms/step
Epoch 7/10
131/131 - 32s - loss: 0.0186 - accuracy: 0.9935 - val_loss: 1.3613 - val_accuracy: 0.8029 - 32s/epoch - 242ms/step
Epoch 8/10
131/131 - 31s - loss: 0.0199 - accuracy: 0.9928 - val_loss: 2.3795 - val_accuracy: 0.7452 - 31s/epoch - 239

In [1]:
# Visualizando os Resultados

